# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
from openai import OpenAI
from IPython.display import display, Markdown, update_display

In [ ]:
# constants
# MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

class LLM_MODEL:

    def ask_model(self, sys_prompt, usr_prompt):
        model_url =  'http://localhost:11434/v1/'
        client = OpenAI(base_url=model_url, api_key='ollama')
        msg = [{'role':'system', 'content':sys_prompt},{'role':'user', 'content':usr_prompt}]
        response = client.chat.completions.create(model=MODEL_LLAMA, messages=msg)
        return response.choices[0].message.content

    def ask_model_stream(self, sys_prompt, usr_prompt):
        model_url =  'http://localhost:11434/v1/'
        client = OpenAI(base_url=model_url, api_key='ollama')
        msg = [{'role':'system', 'content':sys_prompt},{'role':'user', 'content':usr_prompt}]
        stream = client.chat.completions.create(model=MODEL_LLAMA, messages=msg, stream=True)
        return stream

model = LLM_MODEL()

In [ ]:
# Task 1: Tight Speed

sys_prompt = 'You are a helpful assistant who helps me understand technical questions.\n'
usr_prompt = 'It takes Alex 2 hours to travel a distance of 3 kms. What is the speed of Alex?'

resp = model.ask_model(sys_prompt, usr_prompt)
display(Markdown(resp))

In [ ]:
# Task 2: Travel the world in X days?

sys_prompt = 'You are a helpful assistant who helps me understand technical questions.\n'
usr_prompt = 'There are many cities in our world. Can you tell me how to travel the whole world in least number of days ?'

resp = model.ask_model(sys_prompt, usr_prompt)
display(Markdown(resp))

In [ ]:
# Task 3: Generate Code for task 4 to scrap some webpages

sys_prompt = 'You are a coding expert who generates python code for given problem.\n'
usr_prompt = 'Given a website URL, I want to a python function to get the contents of the webpage, and another function to parse all links in the given webpage text.'

resp = model.ask_model(sys_prompt, usr_prompt)
print(resp)

In [ ]:
# Scrap some webpages

import requests
from bs4 import BeautifulSoup

def get_webpage_content(url):
    """
    Fetches the contents of a website.
    
    Args:
        url (str): URL of the webpage.
    
    Returns:
        str: HTML content of the webpage.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching webpage: {e}")
        return None

def parse_links(html_content, base_url=""):
    """
    Parses links from a given HTML content.
    
    Args:
        html_content (str): HTML content of the webpage.
        base_url (str): Base URL to construct relative link URLs. Defaults to "".
    
    Returns:
        list: List of extracted URLs.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    links = []

    for tag in soup.find_all('a'):
        href = tag.get('href')

        # Handle absolute and relative URLs
        if not href or href.startswith('/'):
            url = ""
        else:
            if 0 and base_url:
                url = f"{base_url}{href}"
            else:
                url = href

        if url.startswith('https:/'):
            links.append(url)

    return links


In [ ]:
# Task 4: Make a brochure using the web-content

# Example usage
webname, url = 'Huggingface', "http://www.huggingface.co"

html_content = get_webpage_content(url)
links = parse_links(html_content, url)

print("Extracted Links:")
content = f'Link:{url} -> Content:{html_content}\n'
for link in links:
    print(link)
    html_content = get_webpage_content(url)
    content += f'Link:{link} -> Content:{html_content}\n'

sys_prompt = 'You are a helpful assistant who helps me create a brochure for a website.\n'
usr_prompt = f'You are given the contents for a few pages for the website of {webname} following next line.\n' + \
             content + \
             'Use this information to give the brochure for this company.\n'

stream = model.ask_model_stream(sys_prompt, usr_prompt)

response = ''
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)
